---
title: "Machine Learning - Tom Mitchell"
author: "richsi"
categories: [Book, Notes]
date: "2025-01-06"
image: "images/thumbnail.jpg"
---

# Introduction

Machine learning is inherently a multidisciplinary field. It draws on results from artificial intelligence, probability, statistics, complexity theory, and many more. 

---

### Well-Posed Learning Problems

> **Definition**: A computer program is said to **learn** from experience $E$ with respect to some class of tasks $T$ and performance measure $P$, if its performance at tasks in $T$, as measured by $P$, improves with experience $E$.

**Robot Driving Learning Problem:**

- Task $T$: driving on public four-lane highways using vision sensors
- Performance measure $P$: average distance traveled before an error
- Training experience $E$: a sequence of images and steering commands recorded while observing a human driver

---

### Designing a Learning System
Consider designing a program to learn to player checkers. Assume no external teacher.

- Task $T$: playing checkers
- Performance measure $P$: percent of games won 
- Training experience $E$: games played against itself

**Choosing the Training Experience**

- One key attribute is whether the training experience provides direct or indirect feedback regarding the choices made by the performance system
  * *Direct* training examples can consist of individual checker board states and the correct move for each
  * *Indirect* training examples may consist of move sequences and their final outcomes
- A second important attribute of the training experience is the degree to which the learner controls the sequence of training examples
- A third important attribute is how well it represents the distribution of examples over which the final system performance $P$ must be measured
  * In practice, it is often necessary to learn from a distribution of examples that is somewhat different from those on which the final system will be evaluated
  * Most current theory of machine learning rests on the assumption that the distribution of training examples is identical to the distribution of test examples

**Choosing the Target Function**

The target function will determine what type of knowledge will be learned and how it will be used by the performance program. Let us assume a target function $V$ that maps any legal board state from the set $B$ to some real value. We intend for this target function $V$ to assign higher scores to better board states

Let us define the target value $V(b)$ for an arbitrary board state $b$ in $B$, as follows:

1. if $b$ is a final board state that is won, then $V(b)$ = 100
2. if $b$ is a final board state that is lost, then $V(b)$ = -100
3. if $b$ is a final board state that is drawn, then $V(b)$ = 0
4. if $b$ is not a final state in the game, then $V(b) = V(b')$ where $b'$ is the best final board state that can be achieved starting from $b$ and playing optimally until the end of the game

However, searching for the optimal line of play, all the way until the end of the game, is not efficiently computable. The goal of learning in this case is to discover an *operational* description of $V$. A description that can be used by the checkers-playing program to evaluate states and select moves within realistic time bounds. Thus, we have reduced the learning task in this case to the problem of discovering an *operational description of the ideal target function* $V$. 

**Choosing a Representation for the Target Function**

After specifying the ideal target function $V$, we must choose a representation that the learning program will use to describe the function $\hat{V}$ that it will learn. The choice of representation involves a crucial tradeoff. On one hand, we wish to pick a very expressive representation to allow represnting as close an approximation as possible to the target function $V$. On the other hand, the more expressive the representation, the more training data the program will require in order to choose among the alternative hypotheses it can represent.

**Choosing a Function Approximation Algorithm**

While it is easy to assign a value to board states that correspond to the end of the game, it is less obvious how to assign training values to the *intermediate* board states that occur before the game's end. The rule for estimating training values is summarized as

> **Rule for estimating training values:** 
>$$
>V_{train}(b) \leftarrow \hat{V}(Successor(b))
>$$

$\hat{V}$ denotes the learner's current approximation to $V$ and $Successor(b)$ denotes the next board state following $b$. Iteratively estimating training values based on estimates of successor state values can be proven to converge toward perfect estimates of $V_{train}$.

**Adjusting the Weights**

First, define the *best fit* to the training data. One common approach is to define the best hypothesis, or set of weights, which minimizes the squared error $E$ between training (real) values and values predicted by hypothesis $\hat{V}$.

>$$
>E \equiv \sum_{\langle b, V_{train}(b) \rangle \in\ training\ examples} (V_{train}(b) - \hat{V}(b))^2
>$$

Thus, we seek the weights, or equivalently the $\hat{V}$, that minimizes $E$ for the observed training examples. Many algorithms will incremently refine the weights as new training examples become avavilable and become rebust to errors in these estimated training values. One example is least mean squares (LMS), which performs stochastic gradient-descent search through the space of hypotheses (weight values) to minimize the squared error $E$.

>**LMS weight update rule**
>
>For each training example $\langle b, V_{train}(b)\rangle$
>
>- Use the current weights to calculate $\hat{V}(b)$
>- For each weight $w_i$, update it as $w_i \leftarrow w_i + \eta (V_{train}(b) - \hat{V}(b))x_i$

Here, $\eta$ is a small constant (e.g., 0.1) that moderates the size of the weight update. For a deeper intuitive understanding, notice that when the error $(V_{train}(b) - \hat{V}(b))$ is zero, the weights are not changed. However, if the is positive, that means $\hat{V}(b)$ is too small and the weight is increased in proportion to the value of its corresponding feature. 